# Creacion de Csv de Nodos para el grafo de recurrencia.

## Importanciones

In [ ]:
import pandas as pd

## Carga del Dataset 

En esta parte cargamos los datasets y añadimos las columnas extras necesarias:
  - conexiones --> dataset_final.csv
  - sitios --> ma_sitios.csv  --> Añadir columna de ubicacion basada en su latitud y longitud

###    Conexiones(Carga y Procesamiento)

In [ ]:
conexiones = pd.DataFrame.from_csv("dataset_final.csv", sep = ",")
conexiones.head(3)

In [ ]:
#conexiones["datetime"] = pd.to_datetime(conexiones["datetime"])
conexiones2 = conexiones[["datetime", "mac_usr", "sitio_id"]]
conexiones2.head(3)

###  Truncar hora al valor de 15 mins más bajo

In [ ]:
def truncarFecha(fecha):
    dia,time = fecha.split(" ")
    hora,minuto,segundo = time.split(":")
    aux = {0: "00", 1: "10", 2: "20", 3:"30", 4:"40", 5:"50"}
    minuto2 = aux[int(minuto)//10]
    return dia + " " + hora +":"+ minuto2 + ":00"


In [ ]:
conexiones2["datetime"] = pd.to_datetime(conexiones2["datetime"].apply(lambda fecha: truncarFecha(fecha)))
conexiones2.head()

## Carga y procesamiento de sitios

In [ ]:
sitios = pd.DataFrame.from_csv("ma_sitios2.csv", sep = ",").reset_index()
sitios.head(3)

In [ ]:
places =[["Urdesa", [[-79.915581,-2.178119], [-79.901204,-2.146938]]],
             ["MallSur", [[-79.902062,-2.232042], [-79.894681,-2.226038]]],
             ["CityMall", [[-79.912663,-2.143936], [-79.905968,-2.137932]]],
             ["AlbanBorja", [[-79.918242,-2.170096], [-79.915066,-2.167523]]],
             ["Samborondon", [[-79.874897,-2.160447], [-79.852066,-2.116704]]],
             ["Aeropuerto", [[-79.886227,-2.152728], [-79.880562,-2.146338]]],
             ["Penias", [[-79.8776014239,-2.1855487464], [-79.873009,-2.175392]]],
             ["Quito", [[-78.586922,-0.395161], [-78.274155,0.021973]]],
             ["CentroConvenciones", [[-79.893479,-2.161755], [-79.883652,-2.154894]]],
             ["HiltonColon", [[-79.8998951912,-2.160929558], [-79.8962903023,-2.1576274229]]],
             ["MallSol", [[-79.8954641819,-2.157053837], [-79.8891341686,-2.1539258197]]],
             ["UniversidadGuayaquil", [[-79.9008607864,-2.1847947751], [-79.8959684372,-2.1791554991]]]
            ]

In [ ]:
def localizar(row, places):
    longitud = float(row["longitud"])
    latitud = float(row["latitud"])

    for placeName, ubicacion in places:
        SW, NE= ubicacion
        if (latitud <= NE[0] and longitud <=NE[1] and latitud >= SW[0] and longitud >= SW[1]):
            return placeName
    return "No ubicado"

In [ ]:
sitios['localizacion'] = sitios[["longitud", "latitud"]].apply(lambda row: localizar(row, places), axis = 1)
sitios.head()

In [ ]:
sitios2 = sitios[["sitio_id", "localizacion"]] 
sitios2.head(3)

# UNIENDO LOS DATAFRAMES 

In [ ]:
datos = pd.merge(conexiones2, sitios2, on = "sitio_id")
datos.head()

In [ ]:
datos2 = datos.groupby(["localizacion", "datetime"])
macsDic = {}
cont = 1
archivoNodos = open("nodos.csv", "w")
archivoNodos.write("id,Mac")
archivo = open("todo.csv", "w")
archivo.write("Mac1,Mac2\n")
lugares = {"todo": archivo}
for nombre, dato in datos2:
    if nombre[0] not in lugares:
        lugares[nombre[0]] =open(nombre[0].lower() +"2.csv", "w")
        lugares[nombre[0]].write("Mac1,Mac2\n")
    aux = lugares[nombre[0]]
    macs = list(set(list(dato["mac_usr"])))
    for i in range(len(macs)):
        for j in range(i+1, len(macs)):
            lista = [macs[i], macs[j]]
            lista.sort()
            mac1,mac2 = lista
            if mac1 not in macsDic:
                macsDic[mac1] = cont
                archivoNodos.write("\n"+str(cont) + "," + mac1)
                cont+=1
                
            if mac2 not in macsDic:
                macsDic[mac2] = cont
                archivoNodos.write("\n"+str(cont) + "," + mac2)
                cont+=1
            
            mac1 = macsDic[mac1]
            mac2 = macsDic[mac2]
            aux.write(str(mac1) + "," + str(mac2) + "\n")
            archivo.write(str(mac1) + "," + str(mac2) + "\n")
            
for k,v in lugares.items():
    v.close()
archivoNodos.close()
"Termino"


In [ ]:
for lugar in lugares:
    arcos = pd.DataFrame.from_csv(lugar.lower() +".csv", sep = ",").reset_index()
    aux = arcos.groupby(["Mac1", "Mac2"]).size().rename("peso").reset_index()
    aux = aux.sort_values("peso", ascending = False)
    aux.to_csv(lugar.lower() +"Peso.csv", sep=',', index = False)
    print(lugar)

In [ ]:
conexiones = pd.DataFrame.from_csv("todoPeso.csv", sep = ",").reset_index()
conexiones.head(10)

In [ ]:
conexiones["peso"].value_counts().plot